In [16]:
import pandas as pd
import requests
import json
import pymysql
import getpass

# Translating `product_catagory_names` from Portuguese to English

How will we use an API to send info, not just recieve?

Tasks
1. Get a list of all `product_catagory_names`s currently in our database
2. Send API request to IBM Watson to translate each word in the list
3. Process the response

## 1. Get the unique product catagories

In [19]:
"""this is a comment
asdjfhjsad
asdkjfnkjasd"""
#sdfkj
#sdlkfj
#slkdfj

'this is a comment'

In [18]:
conn = pymysql.connect(host="localhost",
                       port=3306,
                       user="root",
                       passwd=getpass.getpass(),
                       db="olist")

········


In [20]:
sql = """
SELECT DISTINCT
    product_category_name
FROM products
WHERE product_category_name IS NOT NULL;
"""

product_categories = pd.read_sql(sql, conn)

In [22]:
product_categories.head()

,product_category_name
0,perfumaria
1,artes
2,esporte_lazer
3,bebes
4,utilidades_domesticas


In [24]:
product_categories = (product_categories
                      .assign(cats=product_categories["product_category_name"]
                              .str.replace("_", " ")))

In [25]:
product_categories.head()

,product_category_name,cats
0,perfumaria,perfumaria
1,artes,artes
2,esporte_lazer,esporte lazer
3,bebes,bebes
4,utilidades_domesticas,utilidades domesticas


## 2. test API requests

In [26]:
auth = ("apikey", getpass.getpass())

········


In [27]:
headers = {"Content-Type":"application/json"}
url = "https://api.eu-de.language-translator.watson.cloud.ibm.com/instances/3fd0629c-22b6-4f28-b90e-5676d0cc30fc/v3/translate?version=2018-05-01"

In [28]:
data = {"text": ["Tudo Bem", "algo mais"], "model_id": "pt-en"}

In [30]:
response = requests.post(url=url,
            headers=headers,
            auth=auth, 
            data=json.dumps(data))

In [32]:
response.json()["translations"]

[{'translation': 'All Right'}, {'translation': 'something more'}]

In [35]:
def translate(to_translate, languages):
    auth = ("apikey", getpass.getpass())
    headers = {"Content-Type":"application/json"}
    url = "https://api.eu-de.language-translator.watson.cloud.ibm.com/instances/3fd0629c-22b6-4f28-b90e-5676d0cc30fc/v3/translate?version=2018-05-01"
    data = {"text": to_translate, "model_id": languages}
    
    response = requests.post(url=url,
                             headers=headers, 
                             auth=auth, 
                             data=json.dumps(data))
    return response.json()["translations"]

In [36]:
translate(["Tudo Bem", "algo mais"], "pt-en")

········


[{'translation': 'All Right'}, {'translation': 'something more'}]

In [37]:
pd.DataFrame(translate(["Tudo Bem", "algo mais"], "pt-en"))

········


,translation
0,All Right
1,something more


## 3. Combine the two components

In [40]:
to_translate = product_categories["cats"].to_list()

In [42]:
translations = translate(to_translate, "pt-en")

········


In [45]:
pd.DataFrame(translations)

,translation
0,perfumery
1,arts
2,sport leisure
3,babies
4,domestical utilities
...,...
68,house comfort 2
69,portable cooking and food preparers
70,safe and servic
71,movable collchao and upholstery


In [48]:
product_categories = product_categories.assign(english_cats=pd.DataFrame(translations)["translation"])

In [50]:
product_categories.head()

,product_category_name,cats,english_cats
0,perfumaria,perfumaria,perfumery
1,artes,artes,arts
2,esporte_lazer,esporte lazer,sport leisure
3,bebes,bebes,babies
4,utilidades_domesticas,utilidades domesticas,domestical utilities
